# Оглавление

1. [Введение](#введение)
2. [Обзор данных](#обзор-данных)
   - 2.1. Количество пациентов/случаев
   - 2.2. Количество признаков
3. [Уникальные значения признаков](#уникальные-значения-признаков)
   - 3.1. Категориальные признаки
   - 3.2. Числовые признаки
4. [Гистограммы распределения числовых признаков](#гистограммы-распределения-числовых-признаков)
5. [Корреляции и простые зависимости](#корреляции-и-простые-зависимости)
6. [Выводы и инсайты](#выводы-и-инсайты)
7. [Анализ текущих проблем модели](#анализ-текущих-проблем-модели)
<br>

---

## Введение

Dataset from : [Antimicrobial resistance data](https://www.kaggle.com/datasets/amritpal333/antimicrobial-resistance-data/code)

На основании представленных данных прогнозирование чувствительности и устойчивости к препаратам основных видов микроорганизмов.

<br>

## Обзор данных

### Количество пациентов/случаев




In [216]:
import pandas as pd
import re
# Список файлов
file_paths = [
    "AMR_datasets/E-coli, Tables 3-1 to 3-7.xlsx",
    "AMR_datasets/Klebsiella, Tables 3-8 to 3-13.xlsx",
    "AMR_datasets/Pseudomonas, Tables 3-14 to 3-20.xlsx",
    "AMR_datasets/Acinetobacter, Tables 3-21 to 3-25.xlsx",
    "AMR_datasets/Streptococcus, Tables 3-26 to 3-28.xlsx",
    "AMR_datasets/Staphylococcus, Table 3-29.xlsx",
    "AMR_datasets/Enterococci, Tables 3-30 and 3-31.xlsx",
]

grand_total_N = 0

for file_path in file_paths:
    excel_file = pd.ExcelFile(file_path)    
    
    df_sheets = pd.read_excel(file_path, sheet_name=None, header=None)

    file_total_N = 0
    
    for sheet_name, df_sheet in df_sheets.items():

        df_sheet = df_sheet.dropna(how='all')
        
        # Ищем строку, которая будет служить header (первая строка, где первый столбец содержит 'Country')
        header_index = df_sheet.apply(lambda row: row.astype(str).str.contains("Country").any(), axis=1).idxmax()
        if not header_index.any():
            continue
    
        df_sheet = pd.read_excel(file_path, sheet_name=sheet_name, header=header_index)
                
        df_sheet = df_sheet.rename(columns={
            'N': 'N_2012',
            '%R ': '%R_2012',
            '(95% CI)': '(95% CI)_2012',
            'N.1': 'N_2013',
            '%R .1': '%R_2013',
            '(95% CI).1': '(95% CI)_2013',
            'N.2': 'N_2014',
            '%R .2': '%R_2014',
            '(95% CI).2': '(95% CI)_2014',
            'N.3': 'N_2015',
            '%R .3': '%R_2015',
            '(95% CI).3': '(95% CI)_2015'
        })

        for column in ['N_2012', 'N_2013', 'N_2014', 'N_2015']:
            if column in df_sheet.columns:
                df_sheet[column] = pd.to_numeric(df_sheet[column], errors='coerce').fillna(0)

                # Исключаем строки, где в колонке 'Country' значение 'EU/EEA (population-weighted mean)'
                filtered_df = df_sheet[~df_sheet['Country'].astype(str).str.contains(
                    r'EU/EEA\s*\(population-\s*weighted\s*mean\)', flags=re.IGNORECASE, na=False)]
                
                for value in filtered_df[column]:
                    file_total_N += value

    print(f"Total N (tested isolates) for file '{file_path}': {file_total_N}")
    
    grand_total_N += file_total_N

# Вывод общей суммы по всем файлам и листам
print(r"Total N (tested isolates) in whole dataset:", grand_total_N)

Total N (tested isolates) for file 'AMR_datasets/E-coli, Tables 3-1 to 3-7.xlsx': 1879550.2999999998
Total N (tested isolates) for file 'AMR_datasets/Klebsiella, Tables 3-8 to 3-13.xlsx': 382431
Total N (tested isolates) for file 'AMR_datasets/Pseudomonas, Tables 3-14 to 3-20.xlsx': 278819.4
Total N (tested isolates) for file 'AMR_datasets/Acinetobacter, Tables 3-21 to 3-25.xlsx': 65352.0
Total N (tested isolates) for file 'AMR_datasets/Streptococcus, Tables 3-26 to 3-28.xlsx': 135002.79999999996
Total N (tested isolates) for file 'AMR_datasets/Staphylococcus, Table 3-29.xlsx': 164239
Total N (tested isolates) for file 'AMR_datasets/Enterococci, Tables 3-30 and 3-31.xlsx': 72163.0
Total N (tested isolates) in whole dataset: 2977557.4999999995



### Количество признаков

1. **Year**  
   - **Type**: Numerical  
   - **Description**: Year of the study or data collection.  

2. **Country**  
   - **Type**: Categorical  
   - **Description**: Country where the data was collected.  

3. **Microorganism**  
   - **Type**: Categorical  
   - **Description**: The type of microorganism.  

4. **Antimicrobial group**  
   - **Type**: Categorical  
   - **Description**: Group of antimicrobial drugs tested against the microorganism.  

5. **N (Total number of invasive isolates tested)**  
   - **Type**: Numerical  
   - **Description**: Total number of microorganism isolates tested for resistance to a specific antimicrobial group.  

6. **%R (Percentage resistance)**  
   - **Type**: Numerical  
   - **Description**: Percentage of microorganism isolates resistant to a specific antimicrobial group.  
   - **Target variable**: Numerical forecast indicator.

7. **95% CI (Confidence interval)**  
   - **Type**: Numerical range with spliting into lower and upper bounds.
   - **Description**: Confidence interval for the percentage resistance (%R), indicating the range of statistical uncertainty.  




## Уникальные значения признаков

### Категориальные признаки

Country : 30

    'Belgium' 'Finland' 'Sweden' 'Ireland' 'Denmark' 'Netherlands'
    'United Kingdom' 'Germany' 'Norway' 'Malta' 'France' 'Austria'
    'Czech Republic' 'Slovakia' 'Portugal' 'Slovenia' 'Spain' 'Hungary'
    'Latvia' 'Bulgaria' 'Italy' 'Romania' 'Cyprus' 'Poland' 'Croatia'
    'Lithuania' 'Greece' 'Estonia' 'Iceland' 'Luxembourg'

<br>

Microorganism : 7

    'Escherichia coli' 'Klebsiella pneumoniae' 'Pseudomonas aeruginosa'
    'Acinetobacter species' 'Streptococcus pneumoniae' 'Staphylococcus aureus' 'Enterococci'

<br>

Antimicrobical group : 13

    'Piperacillin+tazobactam' 'Ceftazidime' 'Fluoroquinolones' 'Aminoglycosides'
    'Carbapenems' 'Meticillin' 'Penicillin' 'Aminopenicillins' 'Gentamicin'
    'Macrolides' 'Vancomycin' 'Third-generation cephalosporins' 'Penicillins'




### Числовые признаки

Year : 4

    '2012' '2013' '2014' '2015'

<br>

N : Finite

    'From 0 to +∞'

<br>

%R : Finite

    'From 0 to 100%'

<br>

95% CI : The size of the interval decreases as the amount of data increases.
    
    'From -∞ to +∞'


## Гистограммы распределения числовых признаков

Гистограммы для числовых признаков с диапазонами значений по оси X и количеством случаев по оси Y.

## Корреляции и простые зависимости

Анализ возможных корреляций и зависимостей между признаками.

## Выводы и инсайты

Заключение, включая важные инсайты, полученные из данных.

## Анализ текущих проблем модели

Unknown column "%bar", which wasn't integrated in model's logic.

"Less than 10 isolates reported, no percentage calculated" countries data wasn't included in total count of tested isolates.

A sheet reviewing the total number of isolates tested and the percentage of resistance to combinations of different antimicrobials were not included.

In counting the number of unique categorial features, countries with low amount of data was not included.

Penicillin and penicillins are different things?
